In [291]:
#pip install geopy

In [61]:
import folium
from streamlit_folium import folium_static
from folium.plugins import MarkerCluster
import pandas as pd
import xml.etree.ElementTree as ET
import urllib.request
from folium.plugins import FastMarkerCluster
from geopy.geocoders import Nominatim
from urllib.request import urlopen
from xml.etree.ElementTree import parse

**Map**

In [50]:
#create Berlin Map
m = folium.Map(location = [52.520008, 13.404954], tiles = "cartodbpositron", zoom_start=10 ) 

In [51]:
#m

**Monuments**

In [52]:
df = pd.read_csv('data/denkmalliste_berlin_092020.csv')

In [53]:
df = df.head(100) #missing longitude, latitude

In [54]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")

In [55]:
df["loc"] = df["Adresse"].apply(geolocator.geocode)
df["point"]= df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)

In [56]:
#df

**Memorial Plaques**

In [62]:
df = pd.DataFrame(columns=['uid', 'name', 'url', 'strasse', 'ortsteil', 'inhalt', 'erlauterung', 'autor', 'lon', 'lat'])
var_url = urlopen('https://gedenktafeln-in-berlin.de/index.php?id=31&type=123')
xmldoc = parse(var_url)

for item in xmldoc.iterfind('item'):
    uid = item.findtext('uid')
    name = item.findtext('Name')
    url = item.findtext('url')
    strasse = item.findtext('strasse')
    ortsteil = item.findtext('ortsteil')
    inhalt = item.findtext('inhalt')
    erlauterung = item.findtext('erlauterung')
    autor = item.findtext('autor')
    #lon and lat are wrong in the file, should be swapped
    lon = item.findtext('latitude')
    lat = item.findtext('longitude')
    df = df.append({'uid': uid, 'name': name, 'url': url, 'strasse': strasse, 'ortsteil': ortsteil, 'inhalt': inhalt, 'erlauterung': erlauterung, 'autor': autor, 'lon': lon, 'lat': lat}, ignore_index=True)

In [63]:
index = df.index
number_of_rows = len(index)
number_of_rows

3364

In [64]:
df.dtypes

uid            object
name           object
url            object
strasse        object
ortsteil       object
inhalt         object
erlauterung    object
autor          object
lon            object
lat            object
dtype: object

In [65]:
mask = (df['name'] != '') & (df['lat'] != '') & (df['lon'] != '')
df = df[mask]

In [66]:
index = df.index
number_of_rows = len(index)
number_of_rows

3329

In [67]:
df['lon'].isna().sum()

0

In [68]:
df = df.dropna()

2021-06-16 21:42:54.293 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


In [69]:
df['lon'].isna().sum()

0

In [70]:
df = df[df['lon'].notnull()].copy()
df['lon'] = df['lon'].astype(float)

df = df[df['lat'].notnull()].copy()
df['lat'] = df['lat'].astype(float)

In [71]:
df.dtypes

uid             object
name            object
url             object
strasse         object
ortsteil        object
inhalt          object
erlauterung     object
autor           object
lon            float64
lat            float64
dtype: object

In [72]:
memorials = df
df_memorials = memorials.head(50)

In [73]:
for i,r in df_memorials.iterrows():
    location = [r["lat"], r["lon"]]
    folium.Marker(location=location, popup = r['name'], tooltip=r['name']).add_to(m)

In [74]:
m

**Another Map**

In [75]:
#create Berlin Map
m2 = folium.Map(location = [52.520008, 13.404954], zoom_start=10,  tiles='CartoDB dark_matter' ) #tiles = "cartodbpositron", 

In [76]:
m2

In [77]:
df_memorials.head(2)

,uid,name,url,strasse,ortsteil,inhalt,erlauterung,autor,lon,lat
1,1442,100 Jahre Kino in Berlin,https://www.gedenktafeln-in-berlin.de/nc/geden...,Berliner Straße 27/28,Pankow,Das Mosaik wurde auf Anregung des Vereins\n„D...,Inschrift der auf den Putz geschraubten Alumin...,Holger Hübner,13.412401,52.563673
2,404,17. Juni 1953,https://www.gedenktafeln-in-berlin.de/nc/geden...,Leipziger Straße 5-7,Mitte,"An dieser Stelle,\nvor dem &quot;Haus der\nMin...",Die Bronzetafel (120x74cm) wurde bei der Bildg...,Holger Hübner,13.382760,52.509750


In [78]:
df_memorials = df_memorials[df_memorials['lon'].notnull()].copy()
df_memorials['lon'] = df_memorials['lon'].astype(float)

df_memorials = df_memorials[df_memorials['lat'].notnull()].copy()
df_memorials['lat'] = df_memorials['lat'].astype(float)

In [79]:
import plotly.express as px

fig = px.scatter_mapbox(df_memorials, lat="lat", lon="lon", hover_name="name", hover_data=["lat", "lon"], color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="dark")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
#st.plotly_chart(fig)

Dash:
https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Dash_Interactive_Graphs/Scatter_mapbox/recycling.py

**Stumbling blocks**

In [80]:
stumbling_blocks = pd.read_csv('data/stumbling_blocks.csv')
stumbling_blocks

,Name,Address,District,Year
0,Kaufhaus Nathan Israel,Spandauer Str. 26-32,Mitte,NaN
1,Kurt Bujakowski,Weigandufer 30,Neukölln,1902.0
2,Bella (Anspach) Schweitzer (geb. Mai),Innsbrucker Str. 32,Schöneberg,1884.0
3,Anna Aaron (geb. Bonwitt),Kulmbacher Str. 7,Wilmersdorf,1876.0
4,Irma Josefa Abarbanell (geb. Stransky),Knesebeckstr. 32,Charlottenburg,1882.0
...,...,...,...,...
8459,Auguste Mathilde Zweig (geb. Emanuel),Fasanenstr. 42,Wilmersdorf,1858.0
8460,Clara Zwillenberg (geb. Singer),Schützenstr. 53,Steglitz,1893.0
8461,Siegmund Zwillenberg,Schützenstr. 53,Steglitz,1892.0
8462,Vera Zwillenberg,Schützenstr. 53,Steglitz,1922.0


In [81]:
geolocator = Nominatim(user_agent="example app")

In [82]:
stumbling_blocks_t = stumbling_blocks.head(5)

In [83]:
stumbling_blocks_t["loc"] = stumbling_blocks_t["Address"].apply(geolocator.geocode)

<ipython-input-83-a99d4144569c>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [84]:
stumbling_blocks["loc"] = stumbling_blocks["Address"].apply(geolocator.geocode)
stumbling_blocks["point"]= stumbling_blocks["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
stumbling_blocks[['lat', 'lon', 'altitude']] = pd.DataFrame(stumbling_blocks['point'].to_list(), index=stumbling_blocks.index)

2021-06-16 21:43:19.160 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Andreasstra%C3%9Fe+34&format=json&limit=1
2021-06-16 21:43:40.331 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Elberfelder+Str.+20&format=json&limit=1
2021-06-16 21:43:45.483 WARNING urllib3.connectionpool: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=Prinzenstr.+40&format=json&limit=

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Hauptstra%C3%9Fe+109&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))